In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [57]:
data = pd.read_csv("UmojaHack#3:Hotspots/train.csv")


In [71]:
df = pd.DataFrame([range(3), [0, np.NaN, 0], [0, 0, np.NaN], range(3), range(3)])
df

,0,1,2
0,0,1.0,2.0
1,0,NaN,0.0
2,0,0.0,NaN
3,0,1.0,2.0
4,0,1.0,2.0


In [72]:
df[2].mean()

1.5

In [66]:
data[data['landcover_7'] == '       ']

,ID,area,date,lat,lon,burn_area,climate_aet,climate_def,climate_pdsi,climate_pet,...,landcover_1,landcover_2,landcover_3,landcover_4,landcover_5,landcover_6,landcover_7,landcover_8,population_density,precipitation
8,8_2000-04-01,8,2000-04-01,26.771,5.051,0.000000,1217.142668,0.0,-137.683723,1217.142668,...,0.0,0.455180,0.0,0.544820,0.0,0.000000,,0.0,1.231987,0.250554
17,17_2000-04-01,17,2000-04-01,24.107,4.871,0.000615,1285.538959,0.0,-299.597339,1285.538959,...,0.0,0.662713,0.0,0.332061,0.0,0.005225,,0.0,2.876581,0.181754


In [41]:
'''
col = ['ID', 'area', 'date', 'lat', 'lon', 'burn_area', 'climate_aet',
       'climate_def', 'climate_pdsi', 'climate_pet', 'climate_pr',
       'climate_ro', 'climate_soil', 'climate_srad', 'climate_swe',
       'climate_tmmn', 'climate_tmmx', 'climate_vap', 'climate_vpd',
       'climate_vs', 'elevation', 'landcover_0', 'landcover_1', 'landcover_2',
       'landcover_3', 'landcover_4', 'landcover_5', 'landcover_6',
       'landcover_7', 'landcover_8', 'population_density', 'precipitation']
'''

In [44]:
'''
s = data.shape[0]
for i in range(s):
    for j in col:
        if(str(data[j][i]).strip() == ''):
            data.at[i, j] = ''
data.to_csv('train1.csv', index = False)
'''

In [47]:
data.isnull().sum()

ID                        0
area                      0
date                      0
lat                       0
lon                       0
burn_area                 0
climate_aet               0
climate_def               0
climate_pdsi              0
climate_pet               0
climate_pr                0
climate_ro                0
climate_soil              0
climate_srad              0
climate_swe               0
climate_tmmn              0
climate_tmmx              0
climate_vap               0
climate_vpd               0
climate_vs                0
elevation                 0
landcover_0               0
landcover_1               0
landcover_2               0
landcover_3               0
landcover_4               0
landcover_5               0
landcover_6               0
landcover_7               2
landcover_8               0
population_density    19517
precipitation             0
dtype: int64

In [29]:
data.dtypes

lat                   float64
lon                   float64
burn_area             float64
climate_aet           float64
climate_def           float64
climate_pdsi          float64
climate_pet           float64
climate_pr            float64
climate_ro            float64
climate_soil          float64
climate_srad          float64
climate_swe           float64
climate_tmmn          float64
climate_tmmx          float64
climate_vap           float64
climate_vpd           float64
climate_vs            float64
elevation             float64
landcover_0           float64
landcover_1           float64
landcover_2           float64
landcover_3           float64
landcover_4           float64
landcover_5           float64
landcover_6           float64
landcover_7           float64
landcover_8           float64
population_density    float64
precipitation         float64
month                  object
year                   object
dtype: object

In [3]:
data = data.drop(['ID', "area"], axis = 1)
data['month'] = data['date']
data['year'] = data['date']
data = data.drop('date', axis=1)

In [4]:

s = data.shape[0]
for i in range(s):
        data.at[i, 'month'] = int(data.at[i, 'month'].split('-')[1])


In [5]:
for i in range(s):
        data.at[i, 'year'] = int(data.at[i, 'year'].split('-')[0])


In [6]:
ave = data['population_density'].mean()
emp = data[data['population_density'].isnull()]
a = []
for index, row in emp.iterrows():
    #a.append(index)
    #data = data.drop(a)
    data.at[index, 'population_density'] = ave

In [7]:
y = data['burn_area']
x = data.drop("burn_area", axis = 1)

In [8]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# Using different models to find the optimal model
#
'''
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


# Creating a list of regressor algorithms to compare with
#
models = [RandomForestRegressor(), GradientBoostingRegressor(), AdaBoostRegressor(), DecisionTreeRegressor(),\
          SVR(), KNeighborsRegressor(), LinearRegression()]


# Creating lists of the algorithms, to store the accuracy scores of each fold
#
RandomForest, GradientBoosting, AdaBoost, DecisionTree, SVR, KNeighbors, Linear= ([] for x in range(7))


# Creating a list containig the list of each algorithm. Created for easy iteration
#
model_list = [RandomForest, GradientBoosting, AdaBoost, DecisionTree, SVR, KNeighbors, Linear]



# Creating a cross validation of 10 folds
#
kfold  = KFold(n_splits=10, random_state=101)


# Iterating through each model and appending the scores of each fold to the appriopriate list
#
for i, j in zip(models, model_list):
  j.extend(list(cross_val_score(i, X, y, scoring = 'neg_mean_squared_error', cv = kfold)))

  
# Creating a function to convert neg_mean_squared_error to a square root
#
def sq(lis):
  new_lis = []
  lis = np.array(lis)
  for i in lis:
    i = np.sqrt(i*-1)
    new_lis.append(i)
  return new_lis


# Creating a dataframe of all the rmses from the iterations for each model
#
rmses = pd.DataFrame({'Fold': np.arange(1, 11), 'RandomForest': sq(RandomForest), 'GradientBoosting': sq(GradientBoosting), 'Adaboost': sq(AdaBoost), 'DecisionTree': sq(DecisionTree),\
                       'SVR': sq(SVR), 'Kneighbors': sq(KNeighbors), 'Linear': sq(Linear)})

# Setting the index
#
rmses.set_index('Fold', inplace = True)


# Calculating the mean and standard deviation rmse of each algorithm
#
rmses.loc['mean'] = rmses.mean()
rmses.loc['std'] = rmses.std()


# Previewing the rmses dataframe
#
rmses
'''

"\nfrom sklearn.model_selection import KFold, cross_val_score\nfrom sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor\nfrom sklearn.tree import DecisionTreeRegressor\nfrom sklearn.svm import SVR\nfrom sklearn.neighbors import KNeighborsRegressor\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.metrics import mean_squared_error\n\n\n# Creating a list of regressor algorithms to compare with\n#\nmodels = [RandomForestRegressor(), GradientBoostingRegressor(), AdaBoostRegressor(), DecisionTreeRegressor(),          SVR(), KNeighborsRegressor(), LinearRegression()]\n\n\n# Creating lists of the algorithms, to store the accuracy scores of each fold\n#\nRandomForest, GradientBoosting, AdaBoost, DecisionTree, SVR, KNeighbors, Linear= ([] for x in range(7))\n\n\n# Creating a list containig the list of each algorithm. Created for easy iteration\n#\nmodel_list = [RandomForest, GradientBoosting, AdaBoost, DecisionTree, SVR, KNeighbors, Linear]

In [9]:
test = pd.read_csv("UmojaHack#3:Hotspots/test.csv")


In [10]:
ave = test['population_density'].mean()
emp = test[test['population_density'].isnull()]
for index, row in emp.iterrows():
    test.at[index, 'population_density'] = ave

In [11]:
test['month'] = test['date']
test['year'] = test['date']
test = test.drop('date', axis=1)
s = test.shape[0]
for i in range(s):
        test.at[i, 'month'] = int(test.at[i, 'month'].split('-')[1])
for i in range(s):
        test.at[i, 'year'] = int(test.at[i, 'year'].split('-')[0])


In [12]:
sub = test.drop(['area', 'ID', 'burn_area'], axis = 1)

In [13]:
# # Import the necessary libraries first
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2, f_regression


# # Feature extraction
# test1 = SelectKBest(score_func=f_regression, k=16)
# fit = test1.fit(x, y)

# # Summarize scores
# np.set_printoptions(precision=3)
# print(fit.scores_)

# features = fit.transform(x)
# # Summarize selected features
# print(features[0:1,:])

In [14]:
ab = [float(i) for i in """5.130e+02 1.247e+05 1.492e+05 5.049e+04 6.953e+04 1.726e+04 1.433e+04
 2.637e+04 1.112e-01 8.256e+04 5.245e+03 1.293e+05 1.145e+05 5.587e+04
 1.246e+04 3.481e+02 1.545e+02 4.972e+04 4.428e+00 5.577e+04 2.329e+02
 7.798e+01 1.285e+02 1.223e+01 4.341e+01 6.507e+04""".split()]

In [15]:
f = x.columns
f

Index(['lat', 'lon', 'climate_aet', 'climate_def', 'climate_pdsi',
       'climate_pet', 'climate_pr', 'climate_ro', 'climate_soil',
       'climate_srad', 'climate_swe', 'climate_tmmn', 'climate_tmmx',
       'climate_vap', 'climate_vpd', 'climate_vs', 'elevation', 'landcover_0',
       'landcover_1', 'landcover_2', 'landcover_3', 'landcover_4',
       'landcover_5', 'landcover_6', 'landcover_7', 'landcover_8',
       'population_density', 'precipitation', 'month', 'year'],
      dtype='object')

In [16]:
f = ['lat', 'climate_aet', 'climate_def', 'climate_pet', 'climate_pr',
       'climate_ro', 'climate_soil', 'climate_srad', 'climate_swe',
       'climate_tmmn', 'climate_tmmx', 'climate_vap', 'climate_vpd',
       'climate_vs', 'elevation', 'landcover_0', 'landcover_1', 'landcover_2',
       'landcover_3', 'landcover_4', 'landcover_5', 'landcover_6',
       'landcover_7', 'landcover_8', 'population_density', 'precipitation']
abc = [(i, j) for i,j in zip(ab,f)]
abc.sort(reverse = True)
abc

[(149200.0, 'climate_def'),
 (129300.0, 'climate_vap'),
 (124700.0, 'climate_aet'),
 (114500.0, 'climate_vpd'),
 (82560.0, 'climate_tmmn'),
 (69530.0, 'climate_pr'),
 (65070.0, 'precipitation'),
 (55870.0, 'climate_vs'),
 (55770.0, 'landcover_4'),
 (50490.0, 'climate_pet'),
 (49720.0, 'landcover_2'),
 (26370.0, 'climate_srad'),
 (17260.0, 'climate_ro'),
 (14330.0, 'climate_soil'),
 (12460.0, 'elevation'),
 (5245.0, 'climate_tmmx'),
 (513.0, 'lat'),
 (348.1, 'landcover_0'),
 (232.9, 'landcover_5'),
 (154.5, 'landcover_1'),
 (128.5, 'landcover_7'),
 (77.98, 'landcover_6'),
 (43.41, 'population_density'),
 (12.23, 'landcover_8'),
 (4.428, 'landcover_3'),
 (0.1112, 'climate_swe')]

In [17]:
drop_col = [j for i, j in abc[-2: ]]
#drop_col.remove('lat')
drop_col

['landcover_3', 'climate_swe']

In [18]:
x = x.drop(drop_col, axis=1)

In [19]:
sub = sub.drop(drop_col, axis=1)

In [20]:
# Using the top three models; XGBoost, Catboost and Gradientboost to train and make predictions
# Creating a list of models to use
models = [RandomForestRegressor()]
model_names = ['random3']


# Using a for loop to create a submission file for each model
#
for model, model_name in zip(models, model_names):
  regressor = model                      # instantiating the model
  regressor.fit(x, y)                    # Training the model
  predictions  = regressor.predict(sub)  # Making predictions
  submission_df = pd.DataFrame({'ID': test.ID, 'burn_area': predictions}) # Creating a submission file
  submission_df.to_csv(model_name + '_baseline.csv', index = False)

In [21]:
f = ['lat', 'lon', 'climate_aet', 'climate_def', 'climate_pdsi',
       'climate_pet', 'climate_pr', 'climate_ro', 'climate_soil',
       'climate_srad', 'climate_swe', 'climate_tmmn', 'climate_tmmx',
       'climate_vap', 'climate_vpd', 'climate_vs', 'elevation', 'landcover_0',
       'landcover_1', 'landcover_2', 'landcover_3', 'landcover_4',
       'landcover_5', 'landcover_6', 'landcover_7', 'landcover_8',
       'population_density', 'precipitation']

In [22]:
targeted = pd.read_csv("random3_baseline.csv")
targeted.head()

,ID,burn_area
0,0_2014-01-01,0.088133
1,1_2014-01-01,0.085796
2,2_2014-01-01,0.116526
3,3_2014-01-01,0.136310
4,4_2014-01-01,0.142671
